#  <span style="color:orange">Regression Tutorial (REG102) - Level Intermediate</span>

**Created using: PyCaret 2.2** <br />
**Date Updated: November 25, 2020**

# 1.0 Tutorial Objective
Welcome to the regression tutorial **(REG102)** - Level Intermediate. This tutorial assumes that you have completed __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__. If you haven't used PyCaret before and this is your first tutorial, we strongly recommend you to go back and progress through the beginner tutorial to understand the basics of working in PyCaret.

In this tutorial we will use the `pycaret.regression` module to learn:

* **Normalization:**  How to normalize and scale the dataset
* **Transformation:**  How to apply transformations that make the data linear and approximately normal
* **Target Transformation:**  How to apply transformations to the target variable
* **Combine Rare Levels:**  How to combine rare levels in categorical features
* **Bin Numeric Variables:**  How to bin numeric variables and transform numeric features into categorical ones using 'sturges' rule
* **Model Ensembling and Stacking:**  How to boost model performance using several ensembling techniques such as Bagging, Boosting, Voting and Generalized Stacking.
* **Experiment Logging:** How to log experiments in PyCaret using MLFlow backend

Read Time : Approx 60 Minutes


## 1.1 Installing PyCaret
If you haven't installed PyCaret yet. Please follow the link to __[Beginner's Tutorial](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ for instructions on how to install pycaret.

## 1.2 Pre-Requisites
- Python 3.6 or greater
- PyCaret 2.0 or greater
- Internet connection to load data from pycaret's repository
- Completion of Regression Tutorial (REG101) - Level Beginner

## 1.3 For Google colab users:
If you are running this notebook on Google colab, run the following code at top of your notebook to display interactive visuals.<br/>
<br/>
`from pycaret.utils import enable_colab` <br/>
`enable_colab()`

## 1.4 See also:
- __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__
- __[Regression Tutorial (REG103) - Level Expert](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Expert%20-%20REG103.ipynb)__

# 2.0 Brief overview of techniques covered in this tutorial
Before we into the practical execution of the techniques mentioned above in Section 1, it is important to understand what are these techniques are and when to use them. More often than not most of these techniques will help linear and parametric algorithms, however it is not surprising to also see performance gains in tree-based models. The Below explanations are only brief and we recommend that you do extra reading to dive deeper and get a more thorough understanding of these techniques.

- **Normalization:** Normalization / Scaling (often used interchangeably with standardization) is used to transform the actual values of numeric variables in a way that provides helpful properties for machine learning. Many algorithms such as Linear Regression, Support Vector Machine and K Nearest Neighbors assume that all features are centered around zero and have variances that are at the same level of order. If a particular feature in a dataset has a variance that is larger in order of magnitude than other features, the model may not understand all features correctly and could perform poorly. __[Read more](https://sebastianraschka.com/Articles/2014_about_feature_scaling.html#z-score-standardization-or-min-max-scaling)__ <br/>
<br/>
- **Transformation:** While normalization transforms the range of data to remove the impact of magnitude in variance, transformation is a more radical technique as it changes the shape of the distribution so that transformed data can be represented by a normal or approximate normal distirbution. In general, you should transform the data if using algorithms that assume normality or a gaussian distribution. Examples of such models are Linear Regression, Lasso Regression and Ridge Regression. __[Read more](https://en.wikipedia.org/wiki/Power_transform)__<br/>
<br/>
- **Target Transformation:** This is similar to the `transformation` technique explained above with the exception that this is only applied to the target variable. __[Read more](https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html)__ to understand the effects of transforming the target variable in regression.<br/>
<br/>
- **Combine Rare Levels:** Sometimes categorical features have levels that are insignificant in the frequency distribution. As such, they may introduce noise into the dataset due to a limited sample size for learning. One way to deal with rare levels in categorical features is to combine them into a new class. <br/>
<br/>
- **Bin Numeric Variables:** Binning or discretization is the process of transforming numerical variables into categorical features. An example would be `Carat Weight` in this experiment. It is a continious distribution of numeric values that can be discretized into intervals. Binning may improve the accuracy of a predictive model by reducing the noise or non-linearity in the data. PyCaret automatically determines the number and size of bins using Sturges rule.  __[Read more](https://www.vosesoftware.com/riskwiki/Sturgesrule.php)__<br/>
<br/>
- **Model Ensembling and Stacking:** Ensemble modeling is a process where multiple diverse models are created to predict an outcome. This is achieved either by using many different modeling algorithms or using different samples of training data sets. The ensemble model then aggregates the predictions of each base model resulting in one final prediction for the unseen data. The motivation for using ensemble models is to reduce the generalization error of the prediction. As long as the base models are diverse and independent, the prediction error of the model decreases when the ensemble approach is used. The two most common methods in ensemble learning are `Bagging` and `Boosting`. Stacking is also a type of ensemble learning where predictions from multiple models are used as input features for a meta model that predicts the final outcome. __[Read more](https://blog.statsbot.co/ensemble-learning-d1dcd548e936)__<br/>
<br/>
- **Tuning Hyperparameters of Ensemblers:** Similar to hyperparameter tuning for a single machine learning model, we will also learn how to tune hyperparameters for an ensemble model.

# 3.0 Dataset for the Tutorial

For this tutorial we will be using the same dataset that was used in __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__.

#### Dataset Acknowledgements:
This case was prepared by Greg Mills (MBA ’07) under the supervision of Phillip E. Pfeifer, Alumni Research Professor of Business Administration. Copyright (c) 2007 by the University of Virginia Darden School Foundation, Charlottesville, VA. All rights reserved.

The original dataset and description can be __[found here.](https://github.com/DardenDSC/sarah-gets-a-diamond)__ 

# 4.0 Getting the Data

You can download the data from the original source __[found here](https://github.com/DardenDSC/sarah-gets-a-diamond)__ and load it using the pandas read_csv function or you can use PyCaret's data respository to load the data using the get_data function (This will require internet connection).

In [1]:
from pycaret.datasets import get_data
dataset = get_data('diamond', profile=False)

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


Notice that when the `profile` parameter is to `True`, it displays a data profile for exploratory data analysis. Several pre-processing steps as discussed in section 2 above will be performed in this experiment based on this analysis. Let's summarize how the profile has helped make critical pre-processing choices with the data.

- **Missing Values:** There are no missing values in the data. However, we still need imputers in our pipeline just in case the new unseen data has missing values (not applicable in this case). When you execute the `setup()` function, imputers are created and stored in the pipeline automatically. By default, it uses a mean imputer for numeric values and a constant imputer for categorical. This can be changed using the `numeric_imputation` and `categorical_imputation` parameters in `setup()`. <br/>
<br/>
- **Combine Rare Levels:** Notice the distribution of the `Clarity` feature in the dataset. It has 7 distinct classes of which `FL` only appears 4 times. Similarly in the `Cut` feature, the `Fair` level only appears `2.1%` of the time in the training dataset. We will use the `combine_rare_categories` parameter in the setup to combine the rare levels. <br/>
<br/>
- **Data Scale / Range:** Notice how the scale / range of `Carat Weight` is significantly different than the `Price` variable. Carat Weight ranges from between 0.75 to 2.91 while Price ranges from 2,184 all the way up to 101,561. We will deal with this problem by using the `normalize` parameter in setup. <br/>
<br/>
- **Target Transformation:** The target variable `Price` is not normally distributed. It is right skewed with high kurtosis. We will use the `transform_target` parameter in the setup to apply a linear transformation on the target variable. `<br/>
<br/>
- **Bin Numeric Features:** `Carat Weight` is the only numeric feature. When looking at its histogram, the distribution seems to have natural breaks. Binning will convert it into a categorical feature and create several levels using sturges' rule. This will help remove the noise for linear algorithms. <br/>
<br/>

In [2]:
#check the shape of data
dataset.shape

(6000, 8)

In order to demonstrate the `predict_model()` function on unseen data, a sample of 600 has been withheld from the original dataset to be used for predictions. This should not be confused with a train/test split as this particular split is performed to simulate a real life scenario. Another way to think about this is that these 600 records were not available at the time when the machine learning experiment was performed.

In [3]:
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

Data for Modeling: (5400, 8)
Unseen Data For Predictions (600, 8)


# 5.0 Setting up Environment in PyCaret

In the previous tutorial __[Regression (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ we learned how to initialize the environment in pycaret using `setup()`. No additional parameters were passed in our last example as we did not perform any pre-processing steps (other than those that are imperative for machine learning experiments which were performed automatically by PyCaret). In this example we will take it to the next level by customizing the pre-processing pipeline using `setup()`. Let's look at how to implement all the steps discussed in section 4 above.

In [4]:
from pycaret.regression import *

In [5]:
exp_reg102 = setup(data = data, target = 'Price', session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95, 
                  bin_numeric_features = ['Carat Weight'],
                  log_experiment = True, experiment_name = 'diamond1') 

,Description,Value
0,session_id,123
1,Target,Price
2,Original Data,"(5400, 8)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3779, 39)"


Note that this is the same setup grid that was shown in __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__. The only difference here is the customization parameters that were passed to `setup()` are now set to `True`. Also notice that the `session_id` is the same as the one used in the beginner tutorial, which means that the effect of randomization is completely isolated. Any improvements we see in this experiment are solely due to the pre-processing steps taken in `setup()` or any other modeling techniques used in later sections of this tutorial.

# 6.0 Comparing All Models

Similar to __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ we will also begin this tutorial with `compare_models()`. We will then compare the below results with the last experiment.

In [6]:
top3 = compare_models(exclude = ['ransac'], n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,708.8055,2492075.9705,1539.2702,0.9754,0.0749,0.0545,2.7800
xgboost,Extreme Gradient Boosting,775.3881,2808497.1625,1640.2397,0.9728,0.0807,0.0589,0.2940
lightgbm,Light Gradient Boosting Machine,766.0853,3116466.5642,1704.0975,0.9704,0.0799,0.0576,0.0460
rf,Random Forest Regressor,850.2817,3268110.2457,1770.5503,0.9686,0.0904,0.0657,0.4640
huber,Huber Regressor,940.6199,3651906.6533,1891.7126,0.9640,0.0972,0.0708,0.1150
ridge,Ridge Regression,952.2530,3846277.3250,1934.6313,0.9624,0.0971,0.0715,0.0180
br,Bayesian Ridge,956.6502,3999159.6452,1967.8153,0.9608,0.0972,0.0716,0.0190
lr,Linear Regression,960.2938,4046533.1500,1978.6945,0.9604,0.0973,0.0717,0.1820
et,Extra Trees Regressor,964.0365,4407851.6806,2061.4017,0.9569,0.1054,0.0758,0.5350
dt,Decision Tree Regressor,1002.3791,4693069.5462,2138.8882,0.9538,0.1086,0.0780,0.0190


Notice that we have used `n_select` parameter within `compare_models`. In last tutorial you have seen that compare_models by default returns the best performing model (single model based on default sort order). However you can use `n_select` parameter to return top N models. In this example `compare_models` has returned Top 3 models.

In [7]:
type(top3)

list

In [8]:
print(top3)

[PowerTransformedTargetRegressor(border_count=254, loss_function='RMSE',
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                random_state=123,
                                regressor=<catboost.core.CatBoostRegressor object at 0x00000170085DCCC0>,
                                task_type='CPU', verbose=False), PowerTransformedTargetRegressor(base_score=None, booster='gbtree',
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, gamma=None, gpu_id=None,
                                importance_type='gain',
                                interaction_constraints=None,
                                learning_rate=None, max_delta_step=None,
                                max_depth=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
          

For the purpose of comparison we will use the `RMSLE` score. Notice how drastically a few of the algorithms have improved after we performed a few pre-processing steps in `setup()`. 
- Linear Regression RMSLE improved from `0.6690` to `0.0973`
- Ridge Regression RMSLE improved from `0.6689` to `0.0971`
- Huber Regression RMSLE improved from `0.4333` to `0.0972`

To see results for all of the models from the previous tutorial refer to Section 7 in __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__.

# 7.0 Create a Model

In the previous tutorial __[Regression (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ we learned how to create a model using the `create_model()` function. Now we will learn about a few other parameters that may come in handy. In this section, we will create all models using 5 fold cross validation. Notice how the `fold` parameter is passed inside `create_model()` to achieve this.

### 7.1 Create Model (with 5 Fold CV)

In [9]:
dt = create_model('dt', fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1011.2651,5098042.2411,2257.8845,0.9540,0.1069,0.0771
1,1002.9714,4289729.8334,2071.1663,0.9570,0.1067,0.0789
2,1002.9420,6074744.0485,2464.6996,0.9385,0.1088,0.0783
3,966.4307,3299744.3840,1816.5199,0.9645,0.1091,0.0787
4,1109.5188,10745091.7087,3277.9707,0.8962,0.1177,0.0802
Mean,1018.6256,5901470.4431,2377.6482,0.9420,0.1098,0.0786
SD,48.0231,2588546.8457,498.2566,0.0244,0.0041,0.0010


### 7.2 Create Model (Metrics rounded to 2 decimals points)

In [10]:
rf = create_model('rf', round = 2)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,827.12,2367714.94,1538.74,0.97,0.09,0.07
1,932.90,6257192.34,2501.44,0.95,0.09,0.06
2,808.00,2380387.73,1542.85,0.98,0.09,0.06
3,829.33,1816996.40,1347.96,0.98,0.09,0.07
4,803.02,2027001.24,1423.73,0.97,0.09,0.07
5,899.67,5551494.15,2356.16,0.95,0.09,0.06
6,899.23,3507186.77,1872.75,0.97,0.09,0.07
7,819.22,2499508.41,1580.98,0.97,0.09,0.07
8,804.13,2966084.29,1722.23,0.97,0.09,0.06
9,880.19,3307536.18,1818.66,0.97,0.10,0.07


Notice how passing the `round` parameter inside `create_model()` has rounded the evaluation metrics to 2 decimals.

### 7.3 Create Model (KNN)

In [11]:
knn = create_model('knn')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,3254.4923,38773273.3698,6226.8189,0.5743,0.3749,0.2306
1,3834.1424,65171546.3255,8072.8896,0.5005,0.4376,0.2928
2,3403.3240,48313575.8656,6950.7968,0.5569,0.4074,0.2705
3,3350.6552,38518982.0900,6206.3663,0.5750,0.3888,0.2577
4,3196.6928,34982366.4238,5914.5893,0.5637,0.4034,0.2820
5,3043.3822,39239011.1734,6264.1050,0.6651,0.3841,0.2501
6,3573.7008,59454145.1936,7710.6514,0.4414,0.4151,0.2603
7,3257.3364,37205003.8114,6099.5905,0.5295,0.4161,0.2676
8,3149.6089,34857672.6574,5904.0387,0.6577,0.3484,0.2388
9,3253.8450,39834155.5362,6311.4305,0.6207,0.4107,0.2881


In [12]:
print(knn)

PowerTransformedTargetRegressor(algorithm='auto', leaf_size=30,
                                metric='minkowski', metric_params=None,
                                n_jobs=-1, n_neighbors=5, p=2,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                regressor=KNeighborsRegressor(algorithm='auto',
                                                              leaf_size=30,
                                                              metric='minkowski',
                                                              metric_params=None,
                                                              n_jobs=-1,
                                                              n_neighbors=5,
                                                              p=2,
                                                              weights='uniform'),
                                weights='unifo

# 8.0 Tune a Model

In the previous tutorial __[Regression (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ we learned how to automatically tune the hyperparameters of a model using pre-defined grids. Here we will introduce the the `n_iter` parameter in `tune_model()`. `n_iter` is the number of iterations within a random grid search. For every iteration, the model randomly selects one value from a pre-defined grid of hyperparameters. By default, the parameter is set to `10` which means there would be a maximum of 10 iterations to find the best value for hyperparameters. Increasing the value may improve the performance but will also increase the training time. See the example below:

In [13]:
tuned_knn = tune_model(knn)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2147.6416,24591079.0657,4958.9393,0.7300,0.3007,0.1553
1,2845.0366,41527649.8073,6444.1950,0.6817,0.3557,0.1988
2,2510.7885,41911556.7573,6473.9136,0.6156,0.3389,0.1781
3,2642.8821,32553599.5501,5705.5762,0.6408,0.3395,0.1820
4,2358.5821,29310501.6435,5413.9174,0.6344,0.3297,0.1848
5,2398.9496,40336126.4484,6351.0729,0.6557,0.3165,0.1773
6,2597.6844,48332856.3524,6952.1836,0.5459,0.3329,0.1673
7,2335.7423,28743079.1841,5361.2572,0.6365,0.3427,0.1759
8,2533.9681,33660808.0841,5801.7935,0.6694,0.2921,0.1576
9,2446.9586,32173337.3718,5672.1546,0.6936,0.3270,0.1961


In [17]:
tuned_knn2 = tune_model(knn, n_iter = 50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2108.3491,23201486.8012,4816.7922,0.7453,0.2893,0.1529
1,2882.1901,42232297.0796,6498.6381,0.6763,0.3694,0.2075
2,2412.5226,39986446.7037,6323.4837,0.6333,0.3299,0.1740
3,2644.6452,32471082.4721,5698.3403,0.6417,0.3415,0.1878
4,2347.7756,29102773.4567,5394.6986,0.6370,0.3421,0.1921
5,2198.6207,27881561.9624,5280.2994,0.7620,0.2958,0.1719
6,2605.5384,47783952.2747,6912.5937,0.5511,0.3354,0.1664
7,2419.6011,28931961.9186,5378.8439,0.6341,0.3511,0.1854
8,2385.9747,28566622.4865,5344.7753,0.7195,0.2798,0.1559
9,2445.4325,32674838.8656,5716.1909,0.6889,0.3329,0.1978


Notice how two tuned K Nearest Neighbors were created based on the `n_iter` parameter. In `tuned_knn`, the `n_iter` parameter is left to the default value and resulted in R2 of `0.6504`. In `tuned_knn2`, the `n_iter` parameter was set to `50` and the R2 improved to `0.6689`. Observe the differences between the hyperparameters of `tuned_knn` and `tuned_knn2` below:

In [18]:
plot_model(tuned_knn, plot = 'parameter')

,Parameters
algorithm,auto
leaf_size,30
metric,minkowski
metric_params,None
n_jobs,-1
n_neighbors,9
p,2
weights,distance
power_transformer_method,box-cox
power_transformer_standardize,True


In [19]:
plot_model(tuned_knn2, plot = 'parameter')

,Parameters
algorithm,auto
leaf_size,30
metric,euclidean
metric_params,None
n_jobs,-1
n_neighbors,6
p,2
weights,distance
power_transformer_method,box-cox
power_transformer_standardize,True


# 9.0 Ensemble a Model

Ensembling is a common machine learning technique used to improve the performance of models (mostly tree based). There are various techniques for ensembling that we will cover in this section. These include Bagging and Boosting __[(Read More)](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)__. We will use the `ensemble_model()` function in PyCaret which ensembles the trained base estimators using the method defined in the `method` parameter.

In [20]:
# lets create a simple dt
dt = create_model('dt')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,990.7311,3452087.6409,1857.9795,0.9621,0.1082,0.0784
1,1003.5928,5974333.4760,2444.2450,0.9542,0.1006,0.0733
2,1072.0395,5624100.1012,2371.5185,0.9484,0.1143,0.0821
3,957.9666,3199436.7363,1788.6969,0.9647,0.1025,0.0770
4,982.4130,4371744.1854,2090.8716,0.9455,0.1189,0.0809
5,932.6466,5733572.2818,2394.4879,0.9511,0.0962,0.0709
6,1024.7338,4130405.4241,2032.3399,0.9612,0.1108,0.0797
7,968.4182,3470199.5490,1862.8472,0.9561,0.1095,0.0801
8,885.1288,2911378.2441,1706.2761,0.9714,0.1021,0.0732
9,1206.1210,8063437.8229,2839.6193,0.9232,0.1233,0.0846


### 9.1 Bagging

In [21]:
bagged_dt = ensemble_model(dt)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,849.4748,3027621.5993,1740.0062,0.9668,0.0915,0.0672
1,933.2433,6195361.4290,2489.0483,0.9525,0.0880,0.0644
2,844.0390,3678395.0132,1917.9142,0.9663,0.0897,0.0651
3,827.7642,1835711.3755,1354.8843,0.9797,0.0886,0.0683
4,830.3127,2144026.7668,1464.2496,0.9733,0.0970,0.0697
5,907.7377,5389385.4023,2321.5050,0.9540,0.0911,0.0654
6,940.6783,4800962.9470,2191.1100,0.9549,0.0936,0.0667
7,856.0980,2812959.1622,1677.1879,0.9644,0.0929,0.0673
8,872.8414,3557086.9912,1886.0241,0.9651,0.0949,0.0667
9,916.7984,3698444.0347,1923.1339,0.9648,0.1045,0.0729


In [22]:
# check the parameter of bagged_dt
print(bagged_dt)

PowerTransformedTargetRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                                     criterion='mse',
                                                                     max_depth=None,
                                                                     max_features=None,
                                                                     max_leaf_nodes=None,
                                                                     min_impurity_decrease=0.0,
                                                                     min_impurity_split=None,
                                                                     min_samples_leaf=1,
                                                                     min_samples_split=2,
                                                                     min_weight_fraction_leaf=0.0,
                                                                     presort='deprecated',
        

Notice how ensembling has improved the `RMSLE` from `0.1082` to `0.0938`. In the above example we have used the default parameters of `ensemble_model()` which uses the `Bagging` method. Let's try `Boosting` by changing the `method` parameter in `ensemble_model()`. See example below: 

### 9.2 Boosting

In [23]:
boosted_dt = ensemble_model(dt, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,886.6468,2552655.2750,1597.7031,0.9720,0.0983,0.0723
1,1025.1924,6731578.5824,2594.5286,0.9484,0.0970,0.0705
2,900.0017,4901816.3895,2214.0046,0.9550,0.0927,0.0676
3,977.9167,3384193.0721,1839.6176,0.9627,0.1025,0.0768
4,949.0684,3782150.8323,1944.7753,0.9528,0.1099,0.0779
5,969.5532,7847973.2617,2801.4234,0.9330,0.0991,0.0691
6,964.5278,3884831.5166,1970.9976,0.9635,0.0971,0.0704
7,846.7762,2174257.2296,1474.5363,0.9725,0.0940,0.0707
8,848.3080,2789793.0065,1670.2673,0.9726,0.0940,0.0682
9,991.1647,4734708.4520,2175.9385,0.9549,0.1116,0.0764


Notice how easy it is to ensemble models in PyCaret. By simply changing the `method` parameter you can do bagging or boosting which would otherwise have taken multiple lines of code. Note that `ensemble_model()` will by default build `10` estimators. This can be changed using the `n_estimators` parameter. Increasing the number of estimators can sometimes improve results. See an example below:

In [24]:
bagged_dt2 = ensemble_model(dt, n_estimators=50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,828.6424,2462713.5776,1569.3035,0.9730,0.0897,0.0660
1,922.2578,6129373.0829,2475.7571,0.9530,0.0875,0.0634
2,811.9583,2403737.9298,1550.3993,0.9780,0.0863,0.0643
3,838.2974,1882830.1096,1372.1626,0.9792,0.0888,0.0682
4,812.0108,2195364.1628,1481.6761,0.9726,0.0936,0.0674
5,895.9697,5568439.3809,2359.7541,0.9525,0.0899,0.0637
6,918.0721,4126036.3621,2031.2647,0.9612,0.0910,0.0659
7,819.7846,2431995.5035,1559.4857,0.9692,0.0898,0.0658
8,812.8457,3082258.8271,1755.6363,0.9697,0.0910,0.0641
9,884.5120,3405388.0629,1845.3694,0.9676,0.1032,0.0712


Notice how increasing the n_estimators parameter has improved the result. The bagged_dt model with the default `10` estimators resulted in a RMSLE of `0.0996` whereas in bagged_dt2 where `n_estimators = 50` the RMSLE improved to `0.0911`.

### 9.3 Blending

Blending is another common technique for ensembling that can be used in PyCaret. It creates multiple models and then averages the individual predictions to form a final prediction. Let's see an example below:

In [25]:
# train individual models to blend
lightgbm = create_model('lightgbm', verbose = False)
dt = create_model('dt', verbose = False)
lr = create_model('lr', verbose = False)

In [26]:
# blend individual models
blender = blend_models(estimator_list = [lightgbm, dt, lr])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,720.6507,1539807.7595,1240.8899,0.9831,0.0791,0.0594
1,826.5039,3809702.4546,1951.8459,0.9708,0.0759,0.0569
2,761.9634,2948787.0835,1717.2033,0.9730,0.0795,0.0599
3,729.2915,1478819.1474,1216.0671,0.9837,0.0760,0.0591
4,689.9662,1592749.4759,1262.0418,0.9801,0.0810,0.0586
5,753.6933,3676542.5375,1917.4312,0.9686,0.0766,0.0554
6,850.0921,2695722.1883,1641.8655,0.9747,0.0864,0.0640
7,773.7834,2209680.2603,1486.4993,0.9721,0.0817,0.0607
8,697.7342,1709748.8518,1307.5737,0.9832,0.0772,0.0567
9,861.3020,3358470.7572,1832.6131,0.9680,0.0925,0.0648


In [27]:
# blend top3 models from compare_models
blender_top3 = blend_models(top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,665.2764,1449989.3481,1204.1550,0.9841,0.0754,0.0550
1,778.9661,4094787.9434,2023.5582,0.9686,0.0723,0.0529
2,695.0246,2477142.5031,1573.8941,0.9773,0.0744,0.0548
3,660.8897,1204855.9417,1097.6593,0.9867,0.0705,0.0538
4,664.6233,1824243.7836,1350.6457,0.9772,0.0743,0.0539
5,746.6407,4458121.5378,2111.4264,0.9620,0.0748,0.0530
6,772.9459,2502601.4480,1581.9613,0.9765,0.0793,0.0577
7,683.3106,1777844.5113,1333.3584,0.9775,0.0742,0.0536
8,642.6102,1387718.0622,1178.0145,0.9864,0.0709,0.0531
9,751.0474,2361295.2683,1536.6507,0.9775,0.0859,0.0598


In [28]:
print(blender_top3.estimators_)

[<catboost.core.CatBoostRegressor object at 0x000001700BAAD2E8>, XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=123, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='auto',
             validate_parameters=1, verbosity=0), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              r

Now that we have created a `VotingRegressor` using the `blend_models()` function. The model returned by the `blend_models` function is just like any other model that you would create using `create_model()` or `tune_model()`. You can use this model for predictions on unseen data using `predict_model()` in the same way you would for any other model.

### 9.4 Stacking

Stacking is another popular technique for ensembling but is less commonly implemented due to practical difficulties. Stacking is an ensemble learning technique that combines multiple models via a meta-model. Another way to think about stacking is that multiple models are trained to predict the outcome and a meta-model is created that uses the predictions from those models as an input along with the original features. The implementation of `stack_models()` is based on Wolpert, D. H. (1992b). Stacked generalization __[(Read More)](https://www.sciencedirect.com/science/article/abs/pii/S0893608005800231)__. 

Let's see an example below using the top 3 models we have obtained from `compare_models`:

In [29]:
stacker = stack_models(top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,635.1220,1287310.2325,1134.5969,0.9859,0.0733,0.0533
1,763.7071,3254094.7466,1803.9110,0.9751,0.0714,0.0525
2,703.8437,3505172.7168,1872.2106,0.9679,0.0739,0.0542
3,658.4916,1187367.5050,1089.6639,0.9869,0.0700,0.0534
4,675.2516,2341786.7037,1530.2897,0.9708,0.0736,0.0532
5,761.4019,6244475.7485,2498.8949,0.9467,0.0734,0.0526
6,746.6047,2217136.2560,1489.0051,0.9792,0.0792,0.0575
7,679.3283,1573010.5196,1254.1972,0.9801,0.0735,0.0540
8,646.6292,1351938.2156,1162.7288,0.9867,0.0700,0.0527
9,763.1126,2524633.9610,1588.9097,0.9760,0.0850,0.0599


By default, the meta model (final model to generate predictions) is Linear Regression. The meta model can be changed using the `meta_model` parameter. See an example below:

In [30]:
xgboost = create_model('xgboost')
stacker2 = stack_models(top3, meta_model = xgboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,758.7384,2291980.7500,1513.9290,0.9748,0.0822,0.0609
1,889.7577,5361943.0000,2315.5869,0.9589,0.0840,0.0608
2,762.8375,3363142.5000,1833.8872,0.9692,0.0800,0.0596
3,795.9018,2089070.7500,1445.3618,0.9770,0.0773,0.0597
4,777.4120,4090615.2500,2022.5270,0.9490,0.0834,0.0596
5,733.5374,2329257.0000,1526.1903,0.9801,0.0754,0.0559
6,838.2933,2618129.5000,1618.0635,0.9754,0.0847,0.0632
7,728.3295,1788938.5000,1337.5121,0.9774,0.0802,0.0600
8,700.6719,1797315.7500,1340.6400,0.9823,0.0743,0.0558
9,805.1788,2665404.5000,1632.6067,0.9746,0.0900,0.0636


Before we wrap up this section, there is another parameter in `stack_models()` that we haven't seen yet called `restack`. This parameter controls the ability to expose the raw data to the meta model. When set to `True`, it exposes the raw data to the meta model along with all the predictions of the base level models. By default it is set to `True`. See the example below with the `restack` parameter changed to `False`.

# 10.0 Experiment Logging

PyCaret 2.0 embeds MLflow Tracking component as a backend API and UI for logging parameters, code versions, metrics, and output files when running your machine learning code and for later visualizing the results. To log your experiments in pycaret simply use log_experiment and experiment_name parameter in the setup function, as we did in this example.

You can start the UI on `localhost:5000`. Simply initiate the MLFlow server from command line or from notebook. See example below:

In [35]:
# to start the MLFlow server from notebook:
!mlflow ui

### Open localhost:5000 on your browser (below is example of how UI looks like)
![title](https://i2.wp.com/pycaret.org/wp-content/uploads/2020/07/classification_mlflow_ui.png?resize=1080%2C508&ssl=1)

# 11.0 Wrap-up / Next Steps?

We have covered a lot of new concepts in this tutorial. Most importantly we have seen how to use exploratory data analysis to customize a pipeline in `setup()` which has improved the results considerably when compared to what we saw earlier in __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__. WWe have also learned how to perform and tune ensembling in PyCaret.

There are however a few more advanced things to cover in `pycaret.regression` which include interpretating more complex tree based models using shapley values, advanced ensembling techniques such as multiple layer stacknet and more pre-processing pipeline methods. We will cover all of this in our next and final tutorial in the `pycaret.regression` series. 

See you at the next tutorial. Follow the link to __[Regression Tutorial (REG103) - Level Expert](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Expert%20-%20REG103.ipynb)__